WordEmbedding 수행했을시 -> 입력차원이 65863,121,100 이기에   
nn.Embedding 대신 -> nn.Linear 로 바꿔야하며

WordEmbedding 미수행시   -> 입력차원이   65863,121 이기에   
nn.Embedding 추가해야한다

그떄그떄 모델 일부코드 변경해서 돌림 
epoch = 10

<a href="https://colab.research.google.com/drive/13eTxQzJaPzpkF2T6DWMdYzJLPQENZRZt" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import numpy as np
import torch.nn as nn
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import get_cosine_schedule_with_warmup

In [ ]:
train_output = torch.load('./drive/MyDrive/appsheet/train_label.pt')
train_input = torch.load('./drive/MyDrive/appsheet/fasttext_train_input.pt')
test_input = torch.load('./drive/MyDrive/appsheet/fast_text_test_input.pt')
vocab_number = 80097
label_number = 7
batch_size = 32
n_hidden = 200

In [ ]:
print(train_input.shape)
print(test_input.shape)

torch.Size([65863, 121, 100])
torch.Size([13491, 121, 100])


In [ ]:
from sklearn.model_selection import train_test_split # 전체 데이터를 학습 데이터와 평가 데이터로 분할

# train/valid 데이터 분할
train_input, valid_input, train_output, valid_output = train_test_split(train_input, train_output, test_size=0.2, stratify=train_output)

In [ ]:
class TextDataset(Dataset):
    def __init__(self,tokens, labels=None ,is_test=False):
        super().__init__()
        self.tokens = tokens
        self.labels = labels
        self.is_test = is_test
    
    def __len__(self):
        return len(self.tokens)    
    
    def __getitem__(self, idx):
        input = self.tokens[idx]
        label = None
        if not self.is_test:
            label = self.labels[idx]
        return (input, label) if not self.is_test else (input)

In [ ]:
dataset_train = TextDataset(tokens=train_input, labels=train_output, is_test=False)
# dataset_valid = TextDataset(tokens=valid_input, labels=valid_output, is_test=False)
dataset_test = TextDataset(tokens=test_input, is_test=True)


loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
# loader_valid = DataLoader(dataset=dataset_valid, batch_size=batch_size, shuffle=True, drop_last=True)
loader_test = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False,drop_last=True) 

In [ ]:
class TextLSTM(nn.Module):
  def __init__(self):
    super(TextLSTM, self).__init__()
    self.embedding = linear = nn.Linear(100, 200, bias=False)
    self.lstm = nn.LSTM(
      input_size=n_hidden, 
      hidden_size=125, 
      num_layers=1,
      dropout=0.5,
      bidirectional=True,
    )
    self.fc = nn.Linear(2*125, label_number)
    
  def forward(self, hidden_and_cell,X):
    X = self.embedding(X)
    X = X.transpose(0,1)
    outputs, (hidden_forward, hidden_backward) = self.lstm(X, hidden_and_cell)
    
    # forward, backward 방향의 hidden state 를 합침
    outputs = torch.cat(
      (hidden_forward[-1], hidden_backward[-1]), dim=1
    )  
    outputs = self.fc(outputs)
    
    return outputs

In [ ]:
#  GPU 장비 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
model = TextLSTM().to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
criterion = nn.CrossEntropyLoss() # 오차 함수 조정 
optimizer = torch.optim.Adam(model.parameters(), lr=0.0006, weight_decay=0.001) 
scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=len(loader_train)*3, 
    num_training_steps=len(loader_train)*20
)

In [ ]:
def get_valid_loss(model, loader_valid, criterion):
  preds_list = []
  true_list = []
  # == [훈련] =========
  # 모델을 훈련 상태로 설정
  epoch_valid_loss = 0 # 에폭별 손실값 초기화(훈련데이터 용)
  
  # 미니배치 단위로 훈련
  hidden = torch.zeros(2, batch_size, 125, requires_grad=True).to(device)
  cell = torch.zeros(2, batch_size, 125, requires_grad=True).to(device)

  for tokens , labels in tqdm(loader_valid):
      tokens = tokens.to(device)
      labels = labels.to(device)
      
      outputs = model((hidden,cell),tokens)

      preds_list.extend(outputs)
      true_list.extend(labels)
      
      loss = criterion(outputs, labels)
      epoch_valid_loss += loss.item()


  return epoch_valid_loss


In [ ]:
def train(model, loader_train,loader_valid ,criterion, optimizer, scheduler=None, epochs=10, save_file='./models/bidirectional_classifier.pth'):
    
    hidden = torch.zeros(2, batch_size, 125, requires_grad=True).to(device)
    cell = torch.zeros(2, batch_size, 125, requires_grad=True).to(device)

    valid_loss_min = np.inf
    # 총 epoch 만큼 반복
    for epoch in range(epochs):
        
        preds_list = []
        true_list = []
        print(f'에폭 [{epoch+1}/{epochs}] \n---------------------')
        # == [훈련] =========
        model.train() # 모델을 훈련 상태로 설정
        epoch_train_loss = 0 # 에폭별 손실값 초기화(훈련데이터 용)
        
        # 미니배치 단위로 훈련
        for tokens , labels in tqdm(loader_train):
            tokens = tokens.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model((hidden,cell),tokens)

            preds_list.extend(outputs)
            true_list.extend(labels)
            
            loss = criterion(outputs, labels)
            epoch_train_loss += loss.item()
            
            loss.backward()
            optimizer.step() 
            
            if scheduler != None: # 스케줄러 학습률 갱신
                scheduler.step()
        
        # tmp_valid_loss = get_valid_loss(model, loader_valid, criterion)
        print(f'\t 훈련 데이터 손실값 : {epoch_train_loss/len(loader_train):.4f}')
        # print(f'\t 유효성 데이터 손실값 : {tmp_valid_loss/len(loader_valid):.4f}')
        torch.save(model.state_dict(), save_file)
        # if tmp_valid_loss <= valid_loss_min:
        #   print(f'\t### valid 데이터 손실값 감소 ({valid_loss_min:.4f} --> {tmp_valid_loss:.4f}). 모델저장')
        # torch.save(model.state_dict(), save_file)
        # alid_loss_min = tmp_valid_loss
        # == [최적 모델 가중치 찾기] ==
        # 현 에폭에서의 검증 데이터 손실값이 지금까지 가장 작다면
            # 현 에폭의 모델 가중치 (현재까지의 최적 모델 가중치 ) 저장
    return torch.load(save_file)

In [ ]:
#torch.cuda.empty_cache()

In [ ]:
model_state_dict = train(
    model=model, loader_train=loader_train,
    loader_valid=loader_train,
    criterion=criterion, optimizer=optimizer,
    scheduler=scheduler, epochs=10,
    save_file=f'./bidirectional_classifier.pth')

에폭 [1/10] 
---------------------


  0%|          | 0/2058 [00:00<?, ?it/s]

	 훈련 데이터 손실값 : 1.2144
에폭 [2/10] 
---------------------


  0%|          | 0/2058 [00:00<?, ?it/s]

	 훈련 데이터 손실값 : 0.8177
에폭 [3/10] 
---------------------


...

	 훈련 데이터 손실값 : 0.6601
에폭 [9/10] 
---------------------


  0%|          | 0/2058 [00:00<?, ?it/s]

	 훈련 데이터 손실값 : 0.6513
에폭 [10/10] 
---------------------


  0%|          | 0/2058 [00:00<?, ?it/s]

	 훈련 데이터 손실값 : 0.6419


In [ ]:
model.eval() # 모델을 평가 상태로 설정
final_preds = [] # 예측값 저장용 리스트 초기화

hidden = torch.zeros(2, batch_size, 125, requires_grad=True).to(device)
cell = torch.zeros(2, batch_size, 125, requires_grad=True).to(device)

with torch.no_grad():
    for tokens in loader_test: # batch 갯수때문에 복수형
      tokens = tokens.to(device)
      outputs = model((hidden,cell),tokens)
      pred = torch.max(outputs.cpu(), dim=1)[1].numpy() # 예측값

      final_preds.extend(pred)

In [ ]:
print(len(final_preds))
print(len(test_input))

13472
13491


In [ ]:
# 마지막 32개에 맞춰서 마지막 테스트 데이터 예측
import numpy as np
last_input = test_input[-32:]

last_input = last_input.to(device)
outputs = model((hidden,cell),last_input)
last_pred = torch.max(outputs.cpu(), dim=1)[1].numpy() # 예측값
print(last_pred)

[3 6 3 6 6 6 2 6 2 6 2 1 2 0 2 1 1 1 2 3 2 4 1 6 6 0 3 3 4 4 2 6]


In [ ]:
final_preds.extend(last_pred[-19:].tolist())

In [ ]:
print(len(final_preds))

13491


In [ ]:
final_preds = np.array(final_preds)

# 배열을 파일로 저장
np.save('./final_output.npy', final_preds)

In [ ]:
final_preds.shape

(13491,)